# **Top 5 Queries** 
## **Question 5 - Chapter 7 (Exercises)**
**Proposition**: Returns a query that transforms yearly order counts into individual records for each year where orders exist.

**View**: The query retrieves data from the dbo.EmpYearOrders view.

**Columns**: The columns in focus are EmployeeId, a transformed column OrderYear (which represents the years 2014, 2015, and 2016), and NumOrders, which indicates the number of orders for each year.

**Predicate**: The query uses UNPIVOT to convert columns for yearly order counts from the dbo.EmpYearOrders table into rows, showing each employee's orders per year as individual records. It focuses only on years where orders were made by filtering out any instances where the order count is zero (WHERE NumOrders > 0), ensuring that the resulting list includes only active order years for each employee.

*This query is special because it utilizes UNPIVOT to transform the result set into a different fashion.*


In [ ]:
USE TSQLV4;

SELECT empid, orderyear, numorders
FROM dbo.EmpYearOrders
UNPIVOT
(
  numorders FOR orderyear IN (cnt2014, cnt2015, cnt2016)
) AS up
WHERE numorders > 0;



USE Northwinds2022TSQLV7;

SELECT EmployeeId, OrderYear, NumOrders
FROM dbo.EmpYearOrders
UNPIVOT
(
  NumOrders FOR OrderYear IN (Cnt2014, Cnt2015, Cnt2016)
) AS up
WHERE NumOrders > 0;


## **Query (Non-Exercises)**
**Proposition**: Returns the percentage of each order's value relative to the total value of all orders and the total value of orders for each customer.

**View**: The query retrieves data from the Sales.OrderValues view.

**Columns**: The columns involved are: OrderId, CustomerId, and Val. The query introduces two computed columns: PctAll, which represents the percentage of an individual order's value against the total value of all orders, and PctCust, which signifies the percentage of an individual order's value against the total value of orders for that specific customer.

**Predicate**: The query calculates two percentages for each order. The first percentage, PctAll, shows how much each order contributes to the total value of all orders. It does this by dividing the value of each order by the sum of all orders' values and then multiplying by 100. The second percentage, PctCust, shows each order's share of the total orders for that particular customer. It's calculated similarly, but the sum is only for the orders of each specific customer, using a PARTITION BY customer ID.

*This query is special because it utilizes the PARTITION BY for data division in regards to CustomerId within the result set.*


In [ ]:
USE TSQLV4;

SELECT orderid, custid, val,
  100. * val / SUM(val) OVER() AS pctall,
  100. * val / SUM(val) OVER(PARTITION BY custid) AS pctcust
FROM Sales.OrderValues;


USE Northwinds2022TSQLV7;

SELECT Orderid, CustomerId, Val,
  100. * val / SUM(Val) OVER() AS PctAll,
  100. * val / SUM(Val) OVER(PARTITION BY CustomerId) AS PctCust
FROM Sales.OrderValues;

## **Query (Non-Exercises)**
**Proposition**: Returns the cumulative total of order values for each employee, month by month.

**View**: The query retrieves data from the Sales.EmpOrders view.

**Columns**: The involved columns are: EmployeeId, OrderMonth, and Val. The query also introduces a computed column, RunVal, which represents the running total of the Val column for each employee, accumulated over the months in ascending order.

**Predicate**: The query employs a window function (SUM() OVER()) to compute the running total (RunVal) of order values (Val) for each employee (EmployeeId), segmented by month (OrderMonth). The running total is calculated by summing up the values from the start (unbounded preceding) up to the current row within each partition defined by EmployeeId, ordered by OrderMonth.

*This query is special because it utilizes unbounded preceding which conveys the starting point of a window is in the first partition row.*

In [ ]:
USE TSQLV4;

SELECT empid, ordermonth, val,
  SUM(val) OVER(PARTITION BY empid
                ORDER BY ordermonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS runval
FROM Sales.EmpOrders;



USE Northwinds2022TSQLV7;

SELECT EmployeeId, OrderMonth, Val,
  SUM(Val) OVER(PARTITION BY EmployeeId
                ORDER BY OrderMonth
                ROWS BETWEEN UNBOUNDED PRECEDING
                         AND CURRENT ROW) AS RunVal
FROM Sales.EmpOrders;


## **Query (Non-Exercises)**
**Proposition**: Returns an analysis of the order quantities by employee, by customer, both individually and in combination, as well as the total quantity of all orders.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The key columns are EmployeeId, CustomerId, and a computed column SumQty, which represents the total quantity of orders in various groupings.

**Predicate**: This complex query is structured in four parts and each part is combined using the UNION ALL operator. The first part calculates the sum of order quantities (SumQty) for each unique combination of employee (EmployeeId) and customer (CustomerId), grouping by these two dimensions.The second part aggregates the total order quantity for each employee across all customers, indicated by grouping by EmployeeId and setting CustomerId as NULL to signify aggregation across all customers for that employee. The third part sums up the order quantities for each customer across all employees, achieved by grouping by CustomerId with EmployeeId set to NULL. The final part provides the total of all order quantities in the dbo.Orders table, with both EmployeeId and CustomerId set to NULL.

*This query is special because it utilizes the UNION ALL operator to combine the result sets of multiple select statements.*


In [ ]:
USE TSQLV4;

SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid, custid

UNION ALL

SELECT empid, NULL, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid

UNION ALL

SELECT NULL, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY custid

UNION ALL

SELECT NULL, NULL, SUM(qty) AS sumqty
FROM dbo.Orders;




USE Northwinds2022TSQLV7;

SELECT EmployeeId, CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY EmployeeId, CustomerId

UNION ALL

SELECT EmployeeId, NULL, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY EmployeeId

UNION ALL

SELECT NULL, CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY CustomerId

UNION ALL

SELECT NULL, NULL, SUM(Qty) AS SumQty
FROM dbo.Orders;

## **Query (Non-Exercises)**
**Proposition**: Returns an aggregation of total order quantities of each employee, each customer, each employee-customer combination, and for all orders together.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The main columns are EmployeeId, CustomerId, and a computed column SumQty, which accumulates the total quantity of orders.

**Predicate**: The query employs the GROUP BY CUBE() clause to aggregate order quantities at various levels of detail (all possible combinations) within the dbo.Orders table. By doing so, it not only calculates the total quantity of orders for each unique employee-customer pair but also sums up orders for each individual employee across all customers, for each customer across all employees, and the grand total of all orders. 

*This query is special because it utilizes the GROUP BY CUBE to establish all possible combinations of the columns preserved inside the extension.*


In [ ]:
USE TSQLV4;

SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY CUBE(empid, custid);


USE Northwinds2022TSQLV7;

SELECT EmployeeId, CustomerId, SUM(qty) AS SumQty
FROM dbo.Orders
GROUP BY CUBE(EmployeeId, CustomerId);

# **Chapter 7 - Beyond the Fundamentals of Querying (Non-Exercise Queries)**
## **Query**

**Proposition**: Returns the total value of all orders and the total value of orders per customer.

**View**: The query retrieves data from the Sales.OrderValues view.

**Columns**: The columns involved are: OrderId, CustomerId, and Val. Additionally, two calculated columns are generated: TotalValue and CustTotalValue.

**Predicate**: The query does two main things with the order values in the Sales.OrderValues table. First, it finds the total value of all orders put together, giving one grand total. Then, it also figures out the total order value for each customer separately, showing how much each customer's orders add up to. This is done by adding up values in two different ways: one without splitting the orders by customer, and another by grouping the orders by each customer.



In [ ]:
USE TSQLV4; 

SELECT orderid, custid, val,
  SUM(val) OVER() AS totalvalue,
  SUM(val) OVER(PARTITION BY custid) AS custtotalvalue
FROM Sales.OrderValues;


USE Northwinds2022TSQLV7;

SELECT OrderId, CustomerId, Val,
  SUM(Val) OVER() AS TotalValue,
  SUM(Val) OVER(PARTITION BY CustomerId) AS CustTotalValue
FROM Sales.OrderValues;


## **Query**
**Proposition**: Returns the total quantity of orders for each combination of employee and customer.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The columns involved are: EmployeeId, CustomerId, and a calculated column, SumQty, which represents the sum of the quantity (qty) of orders for each employee-customer pair.

**Predicate**: The query aggregates the total quantity of orders (SumQty) for each unique pair of employee (EmployeeId) and customer (CustomerId) in the dbo.Orders table. This is achieved by summing the quantity (qty) column for each grouping of EmployeeId and CustomerId, effectively providing a total order quantity for each employee-customer combination. The GROUP BY clause ensures that the sums are calculated for each distinct pair.






In [ ]:
USE TSQLV4;

SELECT empid, custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid, custid;


USE Northwinds2022TSQLV7;

SELECT EmployeeId, CustomerId, SUM(qty) AS SumQty
FROM dbo.Orders
GROUP BY EmployeeId, CustomerId;

## **Query**
**Proposition**: Returns the total quantity of orders for each employee.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The columns involved are: EmployeeId, and a calculated column, SumQty, which represents the sum of the quantity (Qty) of orders for each employee.

**Predicate**: The query sums up the quantity of orders (Qty) for each employee (EmployeeId) in the dbo.Orders table. This aggregation is done by grouping the results by EmployeeId, which allows the query to calculate the total quantity of orders handled by each employee. The GROUP BY clause ensures that the sum (SumQty) is computed for each unique EmployeeId.



In [ ]:
USE TSQLV4;

SELECT empid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY empid;


USE Northwinds2022TSQLV7;

SELECT EmployeeId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY EmployeeId;

## **Query**
**Proposition**: Returns the total quantity of orders for each customer.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The key columns are: CustomerId, and a computed column, SumQty, which tallies the sum of the order quantities (Qty) per customer.

**Predicate**: This query aggregates the total quantity of orders (represented by Qty) for every customer (CustomerId) listed in the dbo.Orders table. By grouping the data by CustomerId, the query computes the sum of order quantities for each unique customer, yielding SumQty. 


In [ ]:
USE TSQLV4;

SELECT custid, SUM(qty) AS sumqty
FROM dbo.Orders
GROUP BY custid;


USE Northwinds2022TSQLV7;

SELECT CustomerId, SUM(Qty) AS SumQty
FROM dbo.Orders
GROUP BY CustomerId;

## **Query**
**Proposition**: Returns the total quantity of all orders.

**Table**: The query retrieves data from the dbo.Orders table.

**Columns**: The key column utilized is Qty, with a calculated column, SumQty, which accumulates the total quantity across all orders.

**Predicate**: This query aggregates the total quantity of all orders in the dbo.Orders table, producing a singular sum value, SumQty. It does this by summing up all values in the Qty column without any grouping, which results in the overall total quantity of orders. 


In [ ]:
USE TSQLV4;

SELECT SUM(qty) AS sumqty
FROM dbo.Orders;

USE Northwinds2022TSQLV7;

SELECT SUM(Qty) AS SumQty
FROM dbo.Orders;